In [1]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import trimesh as tm 
import tkinter as tk
from tkinter import filedialog 
import pandas as pd
import os
import seaborn as sns
from utilities import select_files, select_file, plot_all

bounding box

In [2]:

box = select_files()
mesh_b = pv.read(box[0].name)
plotter = pv.Plotter()
_ = plotter.add_mesh(mesh_b, opacity=0.70)
#mesh_holes['Stress GPa'] = sim_data['ver_stress']
#plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:62251/index.html?ui=P_0x225484bc9e0_0&reconnect=auto" class="pyvis…

In [3]:
voxels = pv.voxelize(mesh_b, density=10)
p = pv.Plotter()
p.add_mesh(voxels, color=True, show_edges=True, opacity=0.5)
p.add_mesh(mesh_b, color="lightblue", opacity=0.3)
p.show()

Widget(value='<iframe src="http://localhost:62251/index.html?ui=P_0x2254d46d9a0_1&reconnect=auto" class="pyvis…

In [4]:
filtered_files = select_file()
sim_files = pd.read_csv(filtered_files.name)


In [5]:
mesh_list = sim_files['sur_mesh']

In [6]:
p1 = pv.Plotter()
for i in range(len(mesh_list)):

    filename = mesh_list[i]
    mesh_i = pv.read(filename)
    p1.add_mesh(mesh_i, opacity=0.10, color=True)
p1.add_mesh(voxels, color=True, show_edges=True, opacity=0.5)
p1.show(cpos='yx' )

Widget(value='<iframe src="http://localhost:62251/index.html?ui=P_0x225500c6a20_2&reconnect=auto" class="pyvis…

In [7]:
print(voxels)

UnstructuredGrid (0x22550047880)
  N Cells:    1680
  N Points:   2184
  X Bounds:   -4.670e+01, 7.330e+01
  Y Bounds:   -1.800e+02, 2.000e+01
  Z Bounds:   -3.563e+00, 6.644e+01
  N Arrays:   2


In [36]:
n = voxels.n_cells
print(n)

1680


In [8]:
voxels.get_cell(0)

Cell (0x22550046620)
  Type:        <CellType.HEXAHEDRON: 12>
  Linear:      True
  Dimension:   3
  N Points:    8
  N Faces:     6
  N Edges:     12
  X Bounds:    -4.670e+01, -3.670e+01
  Y Bounds:    -1.800e+02, -1.700e+02
  Z Bounds:    -3.563e+00, 6.437e+00

In [21]:
cell_0 = voxels.get_cell(0)
cell_01 = voxels.extract_cells(555)

In [25]:
m = pv.read(mesh_list[0])
intersection, s1_split, s2_split = m.intersection(voxels.extract_cells(555))

ERROR:root:Input for connection index 0 on input port index 1 for algorithm vtkIntersectionPolyDataFilter (00000225D75509E0) is of type vtkUnstructuredGrid, but a vtkPolyData is required.


Convert voxel to polydata(mesh like) structure to use intersection and volume methods

In [31]:
voxel_polydata = cell_01.extract_surface().triangulate()
intersection, s1_split, s2_split = m.intersection(voxel_polydata)
intersection_bool = voxel_polydata.boolean_intersection(m)

In [39]:
print(intersection_bool.points)

[[  -6.7039833 -120.          16.436998 ]
 [ -16.703983  -120.          26.432093 ]
 [ -16.703983  -120.          16.436998 ]
 [ -16.700638  -120.          26.433653 ]
 [  -6.7039833 -120.          26.436998 ]
 [ -16.693457  -120.          26.436998 ]
 [  -7.0060153 -119.788895    16.436998 ]
 [  -6.9172587 -119.78673     16.436998 ]
 [  -8.421813  -119.840416    16.436998 ]
 [  -8.6596985 -119.846725    16.436998 ]
 [  -9.018234  -119.859406    16.436998 ]
 [ -10.362341  -119.88914     16.436998 ]
 [ -12.122328  -119.954796    16.436998 ]
 [ -12.199137  -119.95667     16.436998 ]
 [ -12.221263  -119.94906     16.436998 ]
 [ -12.337474  -119.90748     16.436998 ]
 [  -6.7039833 -119.78152     16.436998 ]
 [ -16.703983  -114.030716    16.436998 ]
 [ -16.703983  -114.03437     16.757618 ]
 [ -16.703983  -113.83263     17.073717 ]
 [ -16.703983  -113.09008     18.166935 ]
 [ -16.703983  -112.8217      18.36518  ]
 [ -16.703983  -112.4431      18.55247  ]
 [ -16.703983  -112.209595    18.6

In [42]:
print(intersection_bool.volume)
print(voxel_polydata.volume)

821.9863731387766
1000.0


In [38]:
#c0 = voxels.get_cell(0).extract_surface().triangulate().boolean_intersection(m)
intersection_bool_0 = voxels.extract_cells(0).extract_surface().triangulate().boolean_intersection(m)

ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
d:\checkouts\ML-Training\ML-env\DS-ML\Lib\site-packages\pyvista\core\filters\poly_data.py:262: UserWarning: Unable to compute boolean intersection when one PolyData is contained within another and no faces intersect.
  warnings.warn(


In [40]:
print(intersection_bool_0.points)

[]


In [26]:
print(intersection.points)

[]


In [34]:
p2 = pv.Plotter()
p2.add_mesh(intersection_bool, color="red", opacity=1)
p2.add_mesh(voxels, color="lightblue", show_edges=True, opacity=0.1)
p2.add_mesh(m,color="green", opacity=0.3)
p2.show()

Widget(value='<iframe src="http://localhost:62251/index.html?ui=P_0x22623c810a0_12&reconnect=auto" class="pyvi…

Compute the intersections and the intersection volume for all voxels
one mesh

In [43]:
def compute_voxel_intersections(surface_mesh, voxel_grid):
    cell_percentage = np.zeros(n) # 
    volume_0 = voxels.extract_cells(0).extract_surface().triangulate().volume
    n = voxel_grid.n_cells
    
    for i in range(n):
        intersection = voxels.extract_cells(i).extract_surface().triangulate().boolean_intersection(surface_mesh)
        if len(intersection.points)>0:
            cell_vol = intersection.volume/volume_0
            cell_percentage[i] = cell_vol
    
    return cell_percentage



In [ ]:
def compute_cell_stress(sim_results, voxel_grid):